# REDES NEURAIS ARTIFICIAIS

[Aula 3 de Deep Learning](http://bit.ly/dn-unb03) da Engenharia de Software da UnB

# Parte 1 - Pré-processamento dos Dados

### Importar as libs

In [22]:
import numpy as np
import pandas as pd
import tensorflow as tf

In [23]:
tf.__version__

'1.14.0'

In [24]:
dataset = pd.read_csv('Churn_Modelling.csv')
dataset.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


### Importar o dataset

In [25]:
X = dataset.iloc[:, 3:13].values
y = dataset.iloc[:, 13].values
print("X >>\n",X)
print("y >>\n",y)

('X >>\n', array([[619, 'France', 'Female', ..., 1, 1, 101348.88],
       [608, 'Spain', 'Female', ..., 0, 1, 112542.58],
       [502, 'France', 'Female', ..., 1, 0, 113931.57],
       ...,
       [709, 'France', 'Female', ..., 0, 1, 42085.58],
       [772, 'Germany', 'Male', ..., 1, 0, 92888.52],
       [792, 'France', 'Female', ..., 1, 0, 38190.78]], dtype=object))
('y >>\n', array([1, 0, 1, ..., 1, 1, 0]))


### Transformando os dados categóricos


In [26]:
# Label Encoding the "Gender" column
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
X[:, 2] = le.fit_transform(X[:, 2])
print(X)
# One Hot Encoding the "Geography" column
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [1])], remainder='passthrough')
X = np.array(ct.fit_transform(X))
print(X)

[[619 'France' 0 ... 1 1 101348.88]
 [608 'Spain' 0 ... 0 1 112542.58]
 [502 'France' 0 ... 1 0 113931.57]
 ...
 [709 'France' 0 ... 0 1 42085.58]
 [772 'Germany' 1 ... 1 0 92888.52]
 [792 'France' 0 ... 1 0 38190.78]]
[[1.0 0.0 0.0 ... 1 1 101348.88]
 [0.0 0.0 1.0 ... 0 1 112542.58]
 [1.0 0.0 0.0 ... 1 0 113931.57]
 ...
 [1.0 0.0 0.0 ... 0 1 42085.58]
 [0.0 1.0 0.0 ... 1 0 92888.52]
 [1.0 0.0 0.0 ... 1 0 38190.78]]


# Dividindo o dataset em conjunto de treinamento e conjunto de teste

In [27]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

# Feature Scaling

In [28]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [29]:
np.shape(X_train)

(8000, 12)

---
# Parte 2 -Vamos construir uma ANN!


### Initializing the ANN

In [30]:
ann = tf.keras.models.Sequential()

### Adding the input layer and the first hidden layer

In [31]:
ann.add(tf.keras.layers.Dense(units=6, activation='sigmoid'))

### Adding the second hidden layer

In [32]:
ann.add(tf.keras.layers.Dense(units=10, activation='relu'))

In [33]:
ann.add(tf.keras.layers.Dense(units=6, activation='relu'))

### Adding the output layer

In [34]:
ann.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

## Part 3 - Training the ANN

### Compiling the ANN

In [35]:
ann.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

### Training the ANN on the Training set

In [36]:
ann.fit(X_train, y_train, batch_size = 32, epochs = 100)

Epoch 1/100
8000/8000 [==============================] - 0s 47us/sample - loss: 0.6189 - acc: 0.6506
Epoch 2/100
8000/8000 [==============================] - 0s 33us/sample - loss: 0.4919 - acc: 0.7960
Epoch 3/100
8000/8000 [==============================] - 0s 32us/sample - loss: 0.4569 - acc: 0.7960
Epoch 4/100
8000/8000 [==============================] - 0s 32us/sample - loss: 0.4296 - acc: 0.7993
Epoch 5/100
8000/8000 [==============================] - 0s 33us/sample - loss: 0.4025 - acc: 0.8155
Epoch 6/100
8000/8000 [==============================] - 0s 31us/sample - loss: 0.3799 - acc: 0.8346
Epoch 7/100
8000/8000 [==============================] - 0s 32us/sample - loss: 0.3702 - acc: 0.8411
Epoch 8/100
8000/8000 [==============================] - 0s 31us/sample - loss: 0.3658 - acc: 0.8444
Epoch 9/100
8000/8000 [==============================] - 0s 32us/sample - loss: 0.3642 - acc: 0.8469
Epoch 10/100
8000/8000 [==============================] - 0s 31us/sample - loss: 0.3626 - a

8000/8000 [==============================] - 0s 41us/sample - loss: 0.3315 - acc: 0.8612
Epoch 82/100
8000/8000 [==============================] - 0s 39us/sample - loss: 0.3320 - acc: 0.8637
Epoch 83/100
8000/8000 [==============================] - 0s 33us/sample - loss: 0.3318 - acc: 0.8639
Epoch 84/100
8000/8000 [==============================] - 0s 31us/sample - loss: 0.3314 - acc: 0.8620
Epoch 85/100
8000/8000 [==============================] - 0s 31us/sample - loss: 0.3311 - acc: 0.8633
Epoch 86/100
8000/8000 [==============================] - 0s 39us/sample - loss: 0.3313 - acc: 0.8616
Epoch 87/100
8000/8000 [==============================] - 0s 41us/sample - loss: 0.3314 - acc: 0.8624
Epoch 88/100
8000/8000 [==============================] - 0s 34us/sample - loss: 0.3306 - acc: 0.8630
Epoch 89/100
8000/8000 [==============================] - 0s 42us/sample - loss: 0.3310 - acc: 0.8641
Epoch 90/100
8000/8000 [==============================] - 0s 37us/sample - loss: 0.3314 - acc: 

# Parte 3 - Fazendo predições e avaliando o modelo

## Prevendo os resultados com o conjunto de testes

In [37]:
y_pred = ann.predict(X_test)
y_pred

array([[0.2739681 ],
       [0.26870203],
       [0.18485999],
       ...,
       [0.06052524],
       [0.13014868],
       [0.23519304]], dtype=float32)

In [38]:
y_pred = (y_pred > 0.9)
y_pred

array([[False],
       [False],
       [False],
       ...,
       [False],
       [False],
       [False]])

# Criando uma Confusion Matrix

In [39]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
print(cm)

[[1593    2]
 [ 333   72]]


# FIM

> Professor Diego Dorgam  
> [@diegodorgam](https://twitter.com/diegodorgam)